# Inference OpenVLA

In [1]:
import os
import pickle
import pandas as pd
import numpy as np
from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForVision2Seq, AutoProcessor
from peft import LoraConfig, get_peft_model

2026-01-13 07:18:12.535855: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-13 07:18:12.576892: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2026-01-13 07:18:12.577007: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2026-01-13 07:18:12.580766: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2026-01-13 07:18:12.590971: I tensorflow/core/platform/cpu_feature_guar

## load VLA

In [2]:
# 1. 모델 로드
vla = AutoModelForVision2Seq.from_pretrained(
    "openvla/openvla-7b",
    attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

config = LoraConfig(
    r=32,                         # Rank
    lora_alpha=64,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"], # Attention 레이어 타겟
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# 3. LoRA 모델로 변환
vla = get_peft_model(vla, config)
print(f"trainable parameters: {vla.print_trainable_parameters()}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
You are attempting to use Flash Attention 2.0 without specifying a torch dtype. This might lead to unexpected behaviour


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


trainable params: 33,554,432 || all params: 7,574,791,616 || trainable%: 0.4430
trainable parameters: None


## load image, prompt and action

In [5]:
df = pd.read_csv('./instruction.csv', )

with open(os.path.join(df["path"].to_list()[0], "policy_out.pkl"), "rb") as f:
    raw_data = pickle.load(f)

actions = [d['actions'] for d in raw_data]
action = actions[0]

im_pth = os.path.join(df["path"].to_list()[0], f"images0/im_{0}.jpg")
image = Image.open(im_pth).convert("RGB")

prompt = "In order to pick up the object, the robot should"

## convert inputs into tensors

In [1]:
vla.eval()
device = "cuda:0"
print(f"device={device}")

processor = AutoProcessor.from_pretrained("openvla/openvla-7b", trust_remote_code=True)

inputs = processor(prompt, image, return_tensors="pt", ).to(device, dtype=torch.bfloat16)
input_ids = inputs["input_ids"].squeeze(0)
labels = torch.full_like(input_ids, -100)

raw_action = np.array(action, dtype=np.float32)
bin_indices = np.clip((raw_action + 1.0) / 2.0 * 255, 0, 255).astype(np.int32)
action_token_ids = torch.tensor(bin_indices + 31000, dtype=torch.long)
labels[-7:] = action_token_ids

NameError: name 'vla' is not defined

## inference

In [ ]:
device = torch.device("cuda")
vla.to(device)
vla.eval()

inputs = {k: v.to(device, dtype=torch.bfloat16 if v.dtype == torch.float32 else v.dtype) for k, v in inputs.items()}

with torch.no_grad():
    action = vla.predict_action(**inputs, unnorm_key="bridge_orig")

print(action)

OutOfMemoryError: CUDA out of memory. Tried to allocate 32.00 MiB. GPU 0 has a total capacity of 7.61 GiB of which 56.19 MiB is free. Process 347964 has 6.45 GiB memory in use. Of the allocated memory 6.31 GiB is allocated by PyTorch, and 39.59 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)